# Gallery

In [ ]:
#| default_exp tutorials/gallery

In [ ]:
#| export
import sys
import math
import numpy as np
from fastcore.basics import patch
import random

In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder
from HexMagic.primitives import MapPath, MapSize, MapRect, MapCord 
from HexMagic.primitives import unique_windy_edge, HexRegion, HexGrid
from HexMagic.terrain import Terrain
from HexMagic.voronoi import generate_plate_terrain, PlateKind, Plate

Terrain.fromSeeds = generate_plate_terrain
from HexMagic.river import River, SoilSystem, ErosionModel
from HexMagic.terrainpatterns import TerrainPatterns

from HexMagic.climate import ClimatePreset, Climate, TerrainFactory
from HexMagic.climate import  Geology
from HexMagic.hydrology import DrainageBasins



In [ ]:
@patch
def dottedClimate(self: Terrain, 
                  flow_levels: int = 5,
                  min_density: float = 0.25,  # Minimum dot density for dry areas
                  debug: bool = False) -> str:
    """Combine climate colors with precipitation-based dot density.

    Args:
        flow_levels: Number of dot density levels (default 5)
        min_density: Minimum density for low-precip areas (0.25 = 25%)
        debug: Print pattern generation info

    Returns:
        SVG string for the overlay
    """

    if 'climate' not in self.fields or 'precipitation' not in self.fields:
        raise ValueError("Need both climate and precipitation computed")

    # Define climate colors (muted palette)
    climate_colors = {
        Climate.MARINE: "#A9B9D4",
        Climate.FRESHWATER: "#7BA3C0",
        Climate.TUNDRA: "#8E7159",
        Climate.DESERT: "#D8A48F",
        Climate.GRASSLAND: "#B9B291",
        Climate.FOREST: "#7A9B76",
        Climate.JUNGLE: "#5D7C5A",
    }

    # Generate patterns for each climate type
    patternGen = TerrainPatterns(self)
    all_patterns = []

    for climate_type in Climate:
        color = climate_colors.get(climate_type, "#cccccc")

        # Create flow_levels density patterns for this climate color
        patterns = patternGen.ballDensity(
            levels=flow_levels + 3,
            fills=[color],
            prefix=f"{climate_type.name.lower()}_ball"
        )[:flow_levels]

        if debug:
            print(f"{climate_type.name}: generated {len(patterns)} patterns with color {color}")

        all_patterns.extend(patterns)

    # Build overlay
    grid = self.hexGrid
    overlay = ""
    used_patterns = set()

    climate_data = self.fields['climate']
    precip_data = self.fields['precipitation']

    # Find max precipitation for normalization
    max_precip = precip_data.max()
    min_precip = precip_data.min()

    if debug:
        print(f"\nPrecipitation range: {min_precip:.0f} - {max_precip:.0f} mm/year")
        print(f"Min density: {min_density * 100}%")

    # Generate hex polygons
    for i in range(len(climate_data)):
        if self.elevations[i] > 0 :
            climate_val = int(climate_data[i])
            precip = precip_data[i]

            # Normalize precipitation to [min_density, 1.0] range
            if max_precip > min_precip:
                normalized_precip = (precip - min_precip) / (max_precip - min_precip)  # [0, 1]
                # Scale to [min_density, 1.0]
                scaled_precip = min_density + normalized_precip * (1.0 - min_density)
                precip_level = int(scaled_precip * (flow_levels - 1))
                precip_level = min(precip_level, flow_levels - 1)
            else:
                precip_level = 0

            # Calculate pattern index: climate_val * flow_levels + precip_level
            pattern_idx = climate_val * flow_levels + precip_level

            # Bounds check
            if pattern_idx >= 0 and pattern_idx < len(all_patterns):
                used_patterns.add(pattern_idx)


                patName = all_patterns[pattern_idx].attributes['id']
                hStyle = StyleCSS(f"url_{patName}",fill = f"url(#{patName})",stroke="gray",stroke_width = 0.5)

                # Generate polygon
                hex_obj = grid.hexes[i].style = hStyle
                self.builder.add_style(hStyle)
                

    # Add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])

    if debug:
        print(f"\nUsed {len(used_patterns)} patterns out of {len(all_patterns)}")

    return grid.styleLayer(f=unique_windy_edge(iterations=2))


## Gallery Item

In [ ]:
#| export
class GalleryItem:

    def __init__(self,name,description,make,width=400,height=400,path:[str] = []):
        self.name = name
        self.description = description
        self.path = path
        self.width = width
        self.height = height
        self.make = make
        self.world = None

    def current(self)->Geology:
        if self.world is None:
            self.world = self.make(self.width,self.height)
        return self.world

In [ ]:
@patch
def elevations(self:GalleryItem):
    world = self.current()
    terrain = world.terrain
    terrain.colorMap()
    sgrid = terrain.hexGrid
    sgrid.builder.adjust("regions", sgrid.styleLayer(f=unique_windy_edge(iterations=2)))
 
    return sgrid.builder.show()

In [ ]:
@patch
def flows(self:GalleryItem):
    world = self.current()
    terrain = world.terrain
    terrain.colorMap()
    sgrid = terrain.hexGrid
    terrain.climate.configure(terrain)
    basin = DrainageBasins(terrain)
    rivers = basin.get_major(5)
    

    river_svg = ""
    for river in rivers:
        river_svg += river.simplify(4).draw(
            min_width=1,
            max_width=9
        )
    
    sgrid.builder.adjust("regions", sgrid.styleLayer(f=unique_windy_edge(iterations=2)))
    sgrid.builder.adjust("rivers",river_svg)
    sgrid.builder.adjust("flows",terrain.flow_diagram())
 
    return sgrid.builder.show()

In [ ]:

@patch
def climate(self:GalleryItem):
    world = self.current()
    terrain = world.terrain
    terrain.colorMap()
    sgrid = terrain.hexGrid
    terrain.climate.configure(terrain)
    basin = DrainageBasins(terrain)
    rivers = basin.get_major(5)
    sgrid.builder.layers = []
    

    river_svg = ""
    for river in rivers:
        river_svg += river.simplify(4).draw(
            min_width=1,
            max_width=9
        )
    terrain.add_climate_overlay(layer_name="climate_precip")
    #sgrid.builder.adjust("regions", sgrid.styleLayer(f=unique_windy_edge(iterations=2)))
    sgrid.builder.adjust("rivers",river_svg)
   
    
 
    return sgrid.builder.show()

In [ ]:
### Items

In [ ]:

def _waterWorld(width,height):
    bounds = MapRect(MapCord(0, 0), MapSize(width, height))
    world = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        ruggedness= 0.2
    )
    terrain = world.terrain
    terrain.elevations = np.zeros(len(terrain.elevations))
    terrain.properties  = []

    return world

giWater = GalleryItem("WaterWorld","Kevin Costner Home",make=_waterWorld,path=["Simple"])

In [ ]:
giWater.elevations()

TypeError: TerrainFactory.create_ocean_world() got an unexpected keyword argument 'ruggedness'

In [ ]:
def _hotSpots(width,height):
   
    world = _waterWorld(width, height)

    terrain = world.terrain
    grid = terrain.hexGrid
    hotSpots = HexRegion(set(),grid)


    for i, plate in enumerate(world.plates):
        if plate.kind != PlateKind.oceanic:
            #for x in plate.region:
            #   terrain.elevations[x] = 50 * (i + 1)
                
            
            volcenter = plate.region.centroid_hex()
            terrain.elevations[volcenter] = 450
            for spot in random.sample(terrain.hexGrid.neighborsOf(volcenter,3),5):
               terrain.elevations += np.maximum(terrain.volcano( spot, 400 , num_rings=4),np.zeros(len(terrain.elevations)))
 
    #world = Geology(terrain,world.plates,age=0.9)
    terrain.carve_to_ocean(0)
    return world

giHot = GalleryItem("Rift world","things collide ",make=_hotSpots,path=["Simple"])

In [ ]:
giHot.flows()

In [ ]:
??HexGrid.neighbors

Object `HexGrid.neighbors` not found.


In [ ]:
??HexGrid


```python
class HexGrid:
    """Hexagonal grid with cube coordinate support."""

    _SQRT3 = math.sqrt(3) 

    def __init__(self, 
                 nRows: int,
                 nCols: int, 
                 radius: float,
                 style: StyleCSS,
                 offset: MapCord = None):
        self.nRows = nRows
        self.nCols = nCols
        self.radius = radius
        self.style = style
        self.offset = offset or MapCord(0, 0)

        self.builder = SVGBuilder()
        self.builder.add_style(style)
        for x in StyleCSS.elevations():
            self.builder.add_style(x)

        self._build_hexes()

    @classmethod
    def from_bounds(cls, bounds: MapRect, radius: float=25, style: StyleCSS=StyleCSS("Hex")) -> 'HexGrid':
        """Create grid to fill a bounding rectangle."""

        nRows = int(bounds.dimensons.height / radius)
        nCols = int(bounds.dimensons.width / radius)
        offset = MapCord(bounds.origin.x - radius, bounds.origin.y - radius)

        return cls(nRows, nCols, radius, style, offset)





    @classmethod
    def centered(cls, rings: int, radius: float, style: StyleCSS, 
                 center: MapCord = None) -> 'HexGrid':
        """Create grid with specified rings around a center point."""
        n = 2 * rings + 1

        # Create grid first with no offset
        grid = cls(nRows=n, nCols=n, radius=radius, style=style, offset=MapCord(0, 0))

        if center is not None:
            # Calculate offset to place middle hex at center
            natural_middle = grid._middle_hex_natural_position()
            grid.offset = MapCord(
                center.x - natural_middle.x,
                center.y - natural_middle.y
            )
            grid._build_hexes()

        return grid

    @property
    def middle(self) -> int:
        """Middle hex index."""
        return (self.nRows // 2) * self.nCols + (self.nCols // 2)

    @property
    def bounds(self) -> MapRect:
        """Bounding rectangle of grid."""
        if not self.hexes:
            return MapRect(MapCord(0, 0), MapSize(0, 0))
        return MapRect(
            self.hexes[0].center,
            MapSize(self.nCols * self.radius * HexGrid._SQRT3, 
                    self.nRows * self.radius * 1.5)
        )


    def text(self, cb=lambda s, i: i):
        i = 0
        for row in range(self.nRows):
            line = "|"
            for col in range(self.nCols):
                line += f" {cb(self, i):5}"
                i += 1
            print(line + " |")

    def rowPartity(self, index):
        "This returns whether a row is even or odd"
        return int(index / self.nCols) % 2

    @property
    def midpoint(self):
        return int(len(self.hexes)/2)
```

**File:** `~/HexMagic/HexMagic/plot/hex.py`

In [ ]:
??TerrainFactory.create_ocean_world


```python
@staticmethod
def create_ocean_world(bounds: MapRect, 
                      preset: str = 'temperate',
                      radius: float = 15,
                      lon_span: float = 10.0,
                      custom_params: dict = None,
                      num_plates=None, 
                      ocean_fraction=0.4,
                      age = 0.25,
                      ruggedness: float = None,
                      factor=1.5, 
                      oceanic_sides=['N'],

                      debug = False) -> Geology:
    """
    Create a blank ocean world ready for terrain generation.

    Args:
        bounds: MapRect for the hex grid
        preset: Climate preset name (see PRESETS)
        radius: Hex radius
        lon_span: Longitude span in degrees (lat span calculated from preset)
        custom_params: Override specific preset parameters

    Returns:
        Terrain with all-ocean elevations and climate parameters set
    """

    if preset not in TerrainFactory.PRESETS:
        raise ValueError(f"Unknown preset: {preset}. Available: {list(TerrainFactory.PRESETS.keys())}")

    climate_preset = TerrainFactory.PRESETS[preset]

    # Calculate geographic bounds
    lat_min, lat_max = climate_preset.lat_range
    lon_min = -lon_span / 2
    lon_max = lon_span / 2

    if ruggedness is None:
        ruggedness = 1.0 - age  # Young terrain is rugged

    slope = int(10 + ruggedness * 30)
    variation = int(20 + ruggedness * 60)

    # Create terrain with all ocean (elevation = 0)
    if num_plates is None:
        terrain = Terrain(bounds, radius=radius)
        terrain.elevations = np.zeros(len(terrain.elevations))  # All ocean
        plates = []
    else:
        terrain, plates = generate_plate_terrain(bounds, 
        radius=radius,slope=slope,variation=variation, 
        num_plates=num_plates, ocean_fraction=ocean_fraction,
        factor=factor, oceanic_sides=oceanic_sides
        )

    # Set geographic bounds
    terrain.geo = GeoBounds(
        lat_min=lat_min,
        lat_max=lat_max,
        lon_min=lon_min,
        lon_max=lon_max
    )

    # Compute hex coordinates
    terrain._compute_hex_coordinates()

    terrain.climate = climate_preset
    world = Geology(terrain,plates=plates,age=age,debug=debug)

    # Apply custom parameter overrides
    if custom_params:
        for key, value in custom_params.items():
            if hasattr(climate_preset, key):
                setattr(climate_preset, key, value)
    if debug:
        print(f"\n=== TERRAIN FACTORY ===")
        print(f"Preset: {climate_preset.name}")
        print(f"Description: {climate_preset.description}")
        print(f"Latitude: {lat_min}° to {lat_max}°")
        print(f"Longitude: {lon_min}° to {lon_max}°")
        print(f"Grid: {terrain.hexGrid.nRows} x {terrain.hexGrid.nCols} hexes")
        print(f"Base temperature range: {climate_preset.base_temp_range[0]}°C to {climate_preset.base_temp_range[1]}°C")
        print(f"Wind: {climate_preset.wind_speed} m/s from {climate_preset.wind_dir}°")

    return world
```

**File:** `~/HexMagic/HexMagic/climate.py`

In [ ]:


def _penWorld(width,height):
    bounds = MapRect(MapCord(0, 0), MapSize(width, height))
    world = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        
        lon_span=5.0,
        num_plates=40,radius=15, ocean_fraction=0.25,factor=6, age=0.9, ruggedness = 0.9, oceanic_sides=['N',"W"]
    )
  

    return world

giPen = GalleryItem("WaterWorld","Kevin Costner Home",make=_penWorld,path=["Simple"])

In [ ]:
giPen.flows()

In [ ]:
def _fooWorld(width,height):
    bounds = MapRect(MapCord(0, 0), MapSize(width, height))
    world = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        
        lon_span=5.0,
        num_plates=40,radius=15, ocean_fraction=0.25,factor=6, age=0.9, ruggedness = 0.9, oceanic_sides=['N',"W"]
    )
    world.terrain.carve_to_ocean(num_lakes=2)
  

    return world

fooPen = GalleryItem("WaterWorld","Kevin Costner Home",make=_fooWorld,path=["Simple"])

In [ ]:
fooPen.flows()


Done at iter 1: 2 lakes


In [ ]:
fooPen.climate()

In [ ]:
fooPen.world.terrain.summarize_climate()


=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████████████  370 ( 54.7%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :     0 (  0.0%)
DESERT      : ██   65 (  9.6%)
GRASSLAND   : █   44 (  6.5%)
FOREST      : ███   90 ( 13.3%)
JUNGLE      : ████  107 ( 15.8%)

=== FIELD STATS ===
Temperature: 23.6°C to 28.6°C (mean: 26.8°C)
Precipitation: 2mm to 5215mm (mean: 1731mm)


In [ ]:
??Terrain.add_climate_overlay


```python
@patch
def add_climate_overlay(self: Terrain, layer_name="climate_precip",debug=False):
    """Combine climate colors with precipitation-based dot density."""

    overlay =  self.dottedClimate(debug=debug)
    self.hexGrid.builder.adjust(layer_name, overlay)

    return self
```

**File:** `~/HexMagic/HexMagic/climate.py`

In [ ]:
??Terrain.dottedClimate


```python
@patch
def dottedClimate(self: Terrain, 
                  flow_levels: int = 5,
                  min_density: float = 0.25,  # Minimum dot density for dry areas
                  debug: bool = False) -> str:
    """Combine climate colors with precipitation-based dot density.

    Args:
        flow_levels: Number of dot density levels (default 5)
        min_density: Minimum density for low-precip areas (0.25 = 25%)
        debug: Print pattern generation info

    Returns:
        SVG string for the overlay
    """

    if 'climate' not in self.fields or 'precipitation' not in self.fields:
        raise ValueError("Need both climate and precipitation computed")

    # Define climate colors (muted palette)
    climate_colors = {
        Climate.MARINE: "#A9B9D4",
        Climate.FRESHWATER: "#7BA3C0",
        Climate.TUNDRA: "#8E7159",
        Climate.DESERT: "#D8A48F",
        Climate.GRASSLAND: "#B9B291",
        Climate.FOREST: "#7A9B76",
        Climate.JUNGLE: "#5D7C5A",
    }

    # Generate patterns for each climate type
    patternGen = TerrainPatterns(self)
    all_patterns = []

    for climate_type in Climate:
        color = climate_colors.get(climate_type, "#cccccc")

        # Create flow_levels density patterns for this climate color
        patterns = patternGen.ballDensity(
            levels=flow_levels + 3,
            fills=[color],
            prefix=f"{climate_type.name.lower()}_ball"
        )[:flow_levels]

        if debug:
            print(f"{climate_type.name}: generated {len(patterns)} patterns with color {color}")

        all_patterns.extend(patterns)

    # Build overlay
    grid = self.hexGrid
    overlay = ""
    used_patterns = set()

    climate_data = self.fields['climate']
    precip_data = self.fields['precipitation']

    # Find max precipitation for normalization
    max_precip = precip_data.max()
    min_precip = precip_data.min()

    if debug:
        print(f"\nPrecipitation range: {min_precip:.0f} - {max_precip:.0f} mm/year")
        print(f"Min density: {min_density * 100}%")

    # Generate hex polygons
    for i in range(len(climate_data)):
        climate_val = int(climate_data[i])
        precip = precip_data[i]

        # Normalize precipitation to [min_density, 1.0] range
        if max_precip > min_precip:
            normalized_precip = (precip - min_precip) / (max_precip - min_precip)  # [0, 1]
            # Scale to [min_density, 1.0]
            scaled_precip = min_density + normalized_precip * (1.0 - min_density)
            precip_level = int(scaled_precip * (flow_levels - 1))
            precip_level = min(precip_level, flow_levels - 1)
        else:
            precip_level = 0

        # Calculate pattern index: climate_val * flow_levels + precip_level
        pattern_idx = climate_val * flow_levels + precip_level

        # Bounds check
        if pattern_idx >= 0 and pattern_idx < len(all_patterns):
            used_patterns.add(pattern_idx)

            patName = all_patterns[pattern_idx].attributes['id']
            fill = f"url(#{patName})"

            # Generate polygon
            hex_obj = grid.hexes[i]
            ret = "<polygon points=\""
            for point in hex_obj.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\"/>"

            overlay += "\t" + ret + "\n"

    # Add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])

    if debug:
        print(f"\nUsed {len(used_patterns)} patterns out of {len(all_patterns)}")

    return overlay
```

**File:** `~/HexMagic/HexMagic/climate.py`